In [1]:
import os
import gspread
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [2]:
def authenticate_drive(credentials_path):
    """Authenticate and return the Google Drive API service."""
    credentials = service_account.Credentials.from_service_account_file(
        credentials_path,
        scopes=['https://www.googleapis.com/auth/drive']
    )
    drive_service = build('drive', 'v3', credentials=credentials)
    return drive_service

In [4]:
"""
def read_excel_file(excel_file_path):
    """
    """Read the Excel file and return a dictionary of email addresses and file IDs."""
    """
    gc = gspread.service_account(filename=excel_file_path)
    sheet = gc.open("G:\My Drive\Job Evalutation Forms\Test File.xlsx").sheet1  # Replace 'Sheet1' with your sheet name
    data = sheet.get_all_records()
    
    # Assuming your Excel file has columns 'Email' and 'File ID'
    email_file_dict = {record['Email']: record['File ID'] for record in data}
    return email_file_dict
"""

# New Work

In [1]:
import pandas as pd

data = pd.read_excel("G:\My Drive\Job Evalutation Forms\Second Emails_Academic.xlsx")

In [2]:
data

,Email,Job Title,Employment Status,file_name
0,jmcfie@strathmore.edu,Senior Lecturer,2 Year Contract,JA - Senior Lecturer.docx
1,gnjenga@strathmore.edu,Senior Lecturer,Permanent Contract,JA - Senior Lecturer.docx
2,bdimba@strathmore.edu,Senior Lecturer,Adjunct Contract,JA - Senior Lecturer.docx
3,jsevilla@strathmore.edu,Senior Lecturer,Permanent Contract,JA - Senior Lecturer.docx
4,momingo@strathmore.edu,Lecturer,3 Year Contract,JA - Lecturer.docx
...,...,...,...,...
138,---,Lecturer,Adjunct Contract,JA - Lecturer.docx
139,gpomondi@strathmore.edu,Lecturer,Adjunct Contract,JA - Lecturer.docx
140,awamburu@strathmore.edu,Lecturer,Adjunct Contract,JA - Lecturer.docx
141,---,Lecturer,Adjunct Contract,JA - Lecturer.docx


In [3]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

def get_gdrive_service():
  creds = None
  if os.path.exists('token.pickle'):
      with open('token.pickle', 'rb') as token:
          creds = pickle.load(token)
  if not creds or not creds.valid:
      if creds and creds.expired and creds.refresh_token:
          creds.refresh(Request())
      else:
          flow = InstalledAppFlow.from_client_secrets_file(
              "client_secret_158237910863-j94jdkja01eh0r5hi7qqm2evhcb2ntgb.apps.googleusercontent.com.json", SCOPES)
          creds = flow.run_local_server(port=0)
      with open('token.pickle', 'wb') as token:
          pickle.dump(creds, token)
  return build('drive', 'v3', credentials=creds)


In [6]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.utils import make_msgid

message = """
Dear Colleague,

I hope the new year has begun on a positive note. 

As a follow up to our communication at the end of last year with regards to reviewing of Job analysis forms, please take a moment to review the Job Analysis form (sent as a separate google form link) to confirm its accuracy in depicting your current job responsibilities. Feel free to provide comments on the form using the "suggesting" function located at the top right of the document.

Your timely feedback is greatly appreciated, and we kindly request that you complete the review by the end of the day on Friday, January 12, 2024.



 """

def send_email(to_address, subject, body, signature_url):
    """
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = "jobevaluation@strathmore.edu"
    msg['To'] = to_address

    body_html = f"{body}<br><img src='{signature_url}'>"
    msg.attach(MIMEText(body_html, 'html'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login("jobevaluation@strathmore.edu", "pbel qjge zwcw plsf") # Use the application-specific password here
    server.send_message(msg)
    server.quit()
 """
    msg = f"Subject: {subject}\n\n{body}"
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login("jobevaluation@strathmore.edu", "pbel qjge zwcw plsf")
    server.sendmail("jobevaluation@strathmore.edu", to_address, msg)
    server.quit()


def search(service, query):
   result = []
   page_token = None
   while True:
       response = service.files().list(q=query, spaces='drive', fields='nextPageToken, files(id, name)', pageToken=page_token).execute()
       for file in response.get('files', []):
           result.append((file.get('id'), file.get('name')))
       page_token = response.get('nextPageToken', None)
       if page_token is None:
           break
   return result

service = get_gdrive_service()
for index, row in data.iterrows():
 email = row['Email']
 file_name = row['file_name']
 file_id = search(service, f"name='{file_name}'")[0][0]
 search_results = search(service, f"name='{file_name}'")
 if search_results:
    file_id = search_results[0][0]
 else:
    print(f"No file found with the name '{file_name}'")
    continue
 file_metadata = {'copyRequiresWriterPermission': False}
 file_metadata = {
   'writersCanInvite': True
}
 service.files().update(fileId=file_id, body=file_metadata).execute()
 service.files().update(fileId=file_id, body=file_metadata).execute()
 service.permissions().create(
     body={
         'role':  'commenter',
         'type': 'user',
         'emailAddress': email,
         'emailMessage':message,
         'withLink': True
     },
     fileId=file_id
 ).execute()


 'signature_url = "https://drive.google.com/file/d/1FdjqLcEJvyoKPlAl7fL0i1AeLDp8BIE1/view?usp=drive_link"'
 signature = """
   
"""
 send_email(email, "Job Analysis Exercise: Review your Role by Friday, January 12, 2024", message,signature)

HttpError: <HttpError 500 when requesting https://www.googleapis.com/drive/v3/files/10EJSdAhhk-eCy-zlLloSeRI-fFU_JAap/permissions?alt=json returned "Internal Error". Details: "[{'message': 'Internal Error', 'domain': 'global', 'reason': 'internalError'}]">

In [25]:
def search(service, query):
   result = []
   page_token = None
   while True:
       response = service.files().list(q=query, spaces='drive', fields='nextPageToken, files(id, name)', pageToken=page_token).execute()
       for file in response.get('files', []):
           result.append((file.get('id'), file.get('name')))
       page_token = response.get('nextPageToken', None)
       if page_token is None:
           break
   return result

service = get_gdrive_service()
for index, row in data.iterrows():
 email = row['Email']
 file_name = row['file_name']
 email = row['Email']
 file_name = row['file_name']
 
 search_results = search(service, f"name='{file_name}'")
 if search_results:
    file_id = search_results[0][0]
    print(f"File found with the name '{file_name}'")
 else:
    print(f"No file found with the name '{file_name}'")
    continue

File found with the name 'JA - Senior Lecturer.docx'
File found with the name 'JA - Senior Lecturer.docx'
File found with the name 'JA - Senior Lecturer.docx'
File found with the name 'JA - Senior Lecturer.docx'
File found with the name 'JA - Lecturer.docx'
File found with the name 'JA - Senior Lecturer.docx'
File found with the name 'JA - Senior Lecturer.docx'
File found with the name 'JA - Lecturer.docx'
File found with the name 'JA - Lecturer.docx'
File found with the name 'JA - Professor.docx'
File found with the name 'JA - Lecturer.docx'
File found with the name 'JA - Lecturer.docx'
File found with the name 'JA - Associate Professor .docx'
File found with the name 'JA - Lecturer.docx'
File found with the name 'JA - Lecturer.docx'
File found with the name 'JA - Senior Lecturer.docx'
File found with the name 'JA - Associate Professor .docx'
File found with the name 'JA - Senior Lecturer.docx'
File found with the name 'JA - Senior Lecturer.docx'
File found with the name 'JA - Senior 

In [11]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os.path
import pickle


# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive']

def main():
   """Shows basic usage of the Drive v3 API.
   Prints the names and ids of the first 10 files the user has access to.
   """
   creds = None
   # The file token.pickle stores the user's access and refresh tokens, and is
   # created automatically when the authorization flow completes for the first
   # time.
   if os.path.exists('token.pickle'):
       with open('token.pickle', 'rb') as token:
           creds = pickle.load(token)
   # If there are no (valid) credentials available, let the user log in.
   if not creds or not creds.valid:
       if creds and creds.expired and creds.refresh_token:
           creds.refresh(Request())
       else:
           flow = InstalledAppFlow.from_client_secrets_file(
               'Evans_client_secret_691790725708-i9sg7796hj0g6rd7kf6ej8c71gigkhsc.apps.googleusercontent.com.json', SCOPES)
           creds = flow.run_local_server(port=0)
       # Save the credentials for the next run
       with open('token.pickle', 'wb') as token:
           pickle.dump(creds, token)

   service = build('drive', 'v3', credentials=creds)

   # Call the Drive v3 API
   results = service.files().list(
       pageSize=10, fields="nextPageToken, files(id, name)").execute()
   items = results.get('files', [])

   if not items:
       print('No files found.')
   else:
       print('Files:')
       for item in items:
           print(u'{0} ({1})'.format(item['name'], item['id']))
           comments = service.comments().list(fileId=item['id'], fields='comments').execute()
           if comments:
               print(f"File {item['name']} has comments.")
           else:
               print(f"File {item['name']} does not have comments.")

if __name__ == '__main__':
   main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=691790725708-i9sg7796hj0g6rd7kf6ej8c71gigkhsc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64403%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=dy42LPUZzRGcV5Pru4sLuHr2G3ytm0&access_type=offline
Files:
ERROR EMAILS  (1bcC_QOss4hJrI3cKUIL1ERSEIzfvBGN4_dSEW6pGIs4)
File ERROR EMAILS  has comments.
drive files.xlsx (1-LnI_1ZkZ18oF9Z5k9yEMWNYzSUwx6M_)
File drive files.xlsx has comments.
Communications officer ; Protocal and events Manager.docx (1_nJOTKapXG2gYwgszSql1CnKuiojNPmd)
File Communications officer ; Protocal and events Manager.docx has comments.
JA - Senior Lecturer.docx (106nxyJbOR7t0YoyaKUHyMsJBjE2O07SF)
File JA - Senior Lecturer.docx has comments.
HRM Systems Comparison (1XKiUKnKMU1GGL2TcxdeSRQRRW1t0iF1AiwVn3_0ElS8)
File HRM Systems Comparison has comments.
JA - Associate Professor .docx (10FdJfBgZNAYxbSGEyqCvuFOWQKSe5wit

In [14]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os.path
import pickle
import pandas as pd

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive']

# Initialize an empty DataFrame
df = pd.DataFrame(columns=["Filename", "Has Comments"])

def main():
  """Shows basic usage of the Drive v3 API.
  Prints the names and ids of the first 10 files the user has access to.
  """
  creds = None
  if os.path.exists('token.pickle'):
      with open('token.pickle', 'rb') as token:
          creds = pickle.load(token)
  if not creds or not creds.valid:
      if creds and creds.expired and creds.refresh_token:
          creds.refresh(Request())
      else:
          flow = InstalledAppFlow.from_client_secrets_file(
              'Evans_client_secret_691790725708-i9sg7796hj0g6rd7kf6ej8c71gigkhsc.apps.googleusercontent.com.json', SCOPES)
          creds = flow.run_local_server(port=0)
      with open('token.pickle', 'wb') as token:
          pickle.dump(creds, token)

  service = build('drive', 'v3', credentials=creds)

  # Call the Drive v3 API
  folder_id = 'your_folder_id' # Replace with your folder ID
  results = service.files().list(
      q=f"'{folder_id}' in parents and mimeType contains '.doc'",
      pageSize=10, 
      fields="nextPageToken, files(id, name, mimeType)"
  ).execute()
  items = results.get('files', [])

  if not items:
      print('No files found.')
  else:
      print('Files:')
      for item in items:
          print(u'{0} ({1})'.format(item['name'], item['id']))
          comments = service.comments().list(fileId=item['id'], fields='comments').execute()
          if comments:
              print(f"File {item['name']} has comments.")
          else:
              print(f"File {item['name']} does not have comments.")
          df = df.append({"Filename": item['name'], "Has Comments": "Yes" if comments else "No"}, ignore_index=True)

if __name__ == '__main__':
  main()

df.to_excel("output.xlsx", index=False)


HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files?q=%27your_folder_id%27+in+parents+and+mimeType+contains+%27.doc%27&pageSize=10&fields=nextPageToken%2C+files%28id%2C+name%2C+mimeType%29&alt=json returned "File not found: .". Details: "[{'message': 'File not found: .', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">